#### Building a spam vs ham filter using NLP with Spark

By: Matt Purvis

##### Importing and Creating Spark Session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

##### Importing data and previewing data

In [0]:
data = spark.sql('select * from smsspamcollection')

In [0]:
data.printSchema()

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)

In [0]:
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

##### Renaming columns

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [0]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

##### Feature Engineering

In [0]:
# Import length to get length of text column
from pyspark.sql.functions import length

In [0]:
# Add length column
data = data.withColumn('length',length(data['text']))

##### Length Investigation

In [0]:
data.groupby('class').mean().show() # Spam texts are typically longer

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

##### More Feature Engineering

In [0]:
# Imports for more features
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

##### Documentation for following objects
Tokenizer - Will take the text and create a list of all the words <br>
Stop_remove - Will remove common meaningless words like 'the' or 'a' <br>
count_vec - Will create a vector of how many times each token appears in the text (bag of words approach) <br>
idf - Inverse Document Frequency - will create a vector for how important the term is in the entire dataset (how many rows have this term in them) <br>
ham_spam_to_numeric - Converts spam column to zeros and ones <br>

In [0]:
tokenizer = Tokenizer(inputCol = 'text',outputCol = 'token_text') 
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'c_vec')
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

##### Transforming features into features vector

In [0]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols = ['tf_idf','length'], outputCol = 'features')

##### Importing and Creating Naive Bayes classifier to use with NLP

In [0]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

##### Creating a pipeline to make all the transformations

In [0]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages= [ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

##### Fitting the pipeline to the data and then transforming the data

In [0]:
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

##### Select the relevant columns (labels and features) to use the naive bayes classifier on

In [0]:
clean_data = clean_data.select('label', 'features')

In [0]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

##### Train/Test Split

In [0]:
training, test = clean_data.randomSplit([.7,.3])

##### Fit/train the naive bayes algorithm on training data

In [0]:
spam_detector = nb.fit(training)

##### Use the model to make predictions on test set

In [0]:
test_results = spam_detector.transform(test)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,7,8...|[-790.52014871620...|[1.0,7.4152820335...| 0.0|
 0.0|(13424,[0,1,2,41,...|[-1062.9742624255...|[1.0,1.9436461190...| 0.0|
 0.0|(13424,[0,1,4,50,...|[-830.87509790989...|[1.0,2.1220398542...| 0.0|
 0.0|(13424,[0,1,5,15,...|[-994.13081278490...|[1.0,7.2525089394...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-541.55055655745...|[1.0,5.6168404918...| 0.0|
 0.0|(13424,[0,1,12,33...|[-439.29179420902...|[1.0,1.2798381646...| 0.0|
 0.0|(13424,[0,1,17,19...|[-803.02882211155...|[1.0,6.2344211313...| 0.0|
 0.0|(13424,[0,1,18,20...|[-850.75888026356...|[1.0,6.9069080257...| 0.0|
 0.0|(13424,[0,1,21,27...|[-1022.7490648299...|[1.0,8.1997820632...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1291.4658679791...|[1.0,2.3406809331...| 0.0|
 0.0|(13424,[0,1,24,31...|[-338.83030822927...|[1.0,5.7079673880...| 0.0|
 0.0|(13424,[0,1,30,12...|[-612.38283344382...|[1.0,1.9124039237...| 0.0|
 0.0|(13424,[0,1,146,1...|[-249.57031827957...|[0.99995741098540...| 0.0|
 0.0|(13424,[0,1,874,1...|[-95.922857573681...|[0.99999997723861...| 0.0|
 0.0|(13424,[0,2,3,4,6...|[-1284.3480818133...|[1.0,2.7973696368...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3409.2890592466...|[1.0,4.9722570757...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3409.2890592466...|[1.0,4.9722570757...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-512.71093186423...|[1.0,9.5575813366...| 0.0|
 0.0|(13424,[0,2,4,8,1...|[-1327.9773215517...|[1.0,4.4376422135...| 0.0|
 0.0|(13424,[0,2,4,128...|[-634.62949184590...|[1.0,1.0511960370...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

##### Evaluate the spam filter using accuracy

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)

In [0]:
# Display the accuracy
print('Accuracy')
acc

Accuracy
Out[68]: 0.9194248085337193

Our spam filter was ~92% accurate! Not bad! Further tweaking could improve the spam filter even more!